***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

# **ETAPA 4 - PROCESSAMENTO PARA CAMADA GOLD**

---
---

<br>

Essa etapa será responsável por mover os dados para a camada gold, agrupando os dados e deixando pronto para uso.

*`Complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br><br>

---
---

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
import uuid
import gc
import urllib.request
from os import listdir
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, TimestampType, LongType, DateType, TimestampType, BooleanType, DoubleType 

### Parte 2 - **Otimizar a Sessão com configurações Personalizadas**

Aqui o será configurado algumas propriedades para que o desempenho da sessão seja mais otimizado 
- Define tamanho fixo de partições para o shuffle para melhorar o paralelismo (usar ***número de partições = número de núclos de CPU * 2 ou 3*** para encontrar melhor cenário possível)
- Define o tamanho máximo de partições para evitar muitos arquivos pequenos
- Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita
- Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados
- Habilita a extensão do Spark SQL para o Delta Lake, permitindo o uso de recursos avançados como ACID transactions, schema enforcement e time travel
- Define o catálogo padrão do Spark como o catálogo Delta, garantindo que operações de leitura, escrita e gerenciamento de tabelas utilizem o engine do Delta Lake


In [0]:
spark = (
    SparkSession.builder
        .appName("Carga Delta")
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.sql.files.maxPartitionBytes", "134217728") 
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()
)

### Parte 3 - **Definindo Origens, Arquivos e Destinos**

`Complete as variáveis:`<br>



In [0]:
nome_datalakehouse = "dataexperts"

nome_camada_silver = "silver"
nome_volume_silver= "vendas" 

nome_camada_gold = "gold"
nome_volume_gold = "vendas" 

O código a seguir armazena em variáveis os caminhos já prontos de origem e de destino dos dados:

In [0]:
origem_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_silver}/{nome_volume_silver}"
destino_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_gold}/{nome_volume_gold}"
destino_tabelas = f"{nome_datalakehouse}.{nome_camada_gold}"

O código a seguir cria o volume de destino caso ele ainda não exista:

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada_gold}.{nome_volume_gold}")

DataFrame[]

### Parte 4 - **Agrupando dados**

O código a seguir tira as colunas de auditoria:

In [0]:
COLUNAS_TECNICAS = [
    "_source_file",
    "_ingestion_date"
]

O código a seguir faz a leitura dos arquivos da camada silver:

In [0]:
def read(volume, file_name):
    try:
        df = spark.read\
            .format("delta")\
            .load(f"{origem_dados}/{file_name}")
        return df
    except Exception as e:
        print(f"Erro ao tentar ler {file_name}: {e}")
        return 0

O código a seguir faz salva as tabelas prontas na camada gold:

In [0]:
def save_table(df, new_file_name):
    try:
        # Remove colunas técnicas
        for col in COLUNAS_TECNICAS:
            if col in df.columns:
                df = df.drop(col)

        if "fato" in new_file_name:
            df_data = spark.read.format("delta").load(f"{destino_dados}/dim_data")

            df_fato = (
                df
                .join(
                    broadcast(df_data.select("sk_data", "ano", "mes")),
                    "sk_data"
                )
            )

            df_fato.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .partitionBy("ano", "mes") \
                .saveAsTable(f"{destino_tabelas}.{new_file_name}")

            df_fato.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .save(f"{destino_dados}/{new_file_name}")

            return df_fato

        else:
            df.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .saveAsTable(f"{destino_tabelas}.{new_file_name}")

            df.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .save(f"{destino_dados}/{new_file_name}")

            return df

    except Exception as e:
        print(f"Ocorreu um erro na função save(): {e}")
        return 0

### Parte 5 - **Utilizando as funções**

In [0]:
new_files_names = [
    "dim_categoria_produto",
    "dim_cliente",
    "dim_data", 
    "dim_localidade",
    "dim_produto",
    "fato_vendas"
] 

files = listdir(f"{origem_dados}")

for i, (file, new_file_name) in enumerate(zip(files,new_files_names)): 

    df_read = read(nome_volume_gold, file)
    df_read = save_table(df_read, new_file_name)

    print(f"Arquivo {file} processado com sucesso.")

Arquivo silver_dim_categoria_produto processado com sucesso.
Arquivo silver_dim_cliente processado com sucesso.
Arquivo silver_dim_data processado com sucesso.
Arquivo silver_dim_localidade processado com sucesso.
Arquivo silver_dim_produto processado com sucesso.
Arquivo silver_fato_vendas processado com sucesso.


### Parte 6 - **Limpeza de Cache e Outros**

O código a seguir libera memória de objetos não mais utilizados:

In [0]:
gc.collect()

22028


<br>

---
---

### **Resultados**

Código simples para mostrar se deu certo ou não essa etapa:

In [0]:
# %skip
new_files_names = [
    "dim_categoria_produto",
    "dim_cliente",
    "dim_data", 
    "dim_localidade",
    "dim_produto",
    "fato_vendas"
] 

def see_tables(new_file_names):
    for file in new_files_names:
        df = spark.read.format("delta").load(f"/Volumes/{nome_datalakehouse}/{nome_camada_gold}/{nome_volume_gold}/{file}")
        display(df)

see_tables(new_files_names)

categoria_id,categoria_nome,sk_categoria
8,ESPORTES,1
11,PAPELARIA,2
2,ELETRODOMÉSTICOS,3
12,PET SHOP,4
1,ELETRÔNICOS,5
10,BRINQUEDOS,6
7,COSMÉTICOS,7
3,ALIMENTOS,8
5,VESTUÁRIO,9
4,BEBIDAS,10


cliente_id,nome_cliente,estado_cliente,cidade_cliente,sk_cliente
271,Cliente 0271,AL,Maceió,1
372,Cliente 0372,TO,Palmas,2
722,Cliente 0722,CE,Juazeiro Do Norte,3
212,Cliente 0212,ES,Vila Velha,4
234,Cliente 0234,BA,Feira De Santana,5
362,Cliente 0362,SP,Sorocaba,6
331,Cliente 0331,PE,Jaboatão Dos Guararapes,7
547,Cliente 0547,SC,Florianópolis,8
459,Cliente 0459,PA,Santarém,9
673,Cliente 0673,CE,Maracanaú,10


data_id,data,ano,mes,dia,dia_semana,final_de_semana,sk_data
20220602,2022-06-02,2022,Junho,2,quinta,false,1
20230818,2023-08-18,2023,Agosto,18,sexta,false,2
20251006,2025-10-06,2025,Outubro,6,segunda,false,3
20240410,2024-04-10,2024,Abril,10,quarta,false,4
20241001,2024-10-01,2024,Outubro,1,terça,false,5
20220717,2022-07-17,2022,Julho,17,domingo,true,6
20230902,2023-09-02,2023,Setembro,2,sábado,true,7
20250310,2025-03-10,2025,Março,10,segunda,false,8
20250626,2025-06-26,2025,Junho,26,quinta,false,9
20240403,2024-04-03,2024,Abril,3,quarta,false,10


localidade_id,estado_venda,cidade_venda,sk_localidade
62,RO,Porto Velho,1
8,BA,Ilhéus,2
38,PB,Campina Grande,3
39,PB,João Pessoa,4
49,PR,Londrina,5
26,MG,Betim,6
33,MT,Cuiabá,7
40,PE,Caruaru,8
11,BA,Vitória Da Conquista,9
71,SC,Chapecó,10


produto_id,preco_lista,sk_produto
271,64.38,1
212,53.59,2
234,18.34,3
290,18.66,4
240,45.22,5
117,26.55,6
184,36.22,7
203,51.93,8
62,24.25,9
263,5.0,10


sk_data,venda_id,quantidade,preco_lista,valor_total,sk_categoria,sk_cliente,sk_produto,sk_localidade,ano,mes
695,271,2,83.72,167.44,10,296,260,55,2022,Setembro
683,1785,1,40.02,40.02,8,402,101,46,2024,Abril
449,1898,2,13.22,26.44,3,598,162,24,2023,Abril
382,2851,1,40.43,40.43,6,56,138,60,2024,Julho
1432,7312,2,48.49,96.98,11,4,208,80,2025,Junho
466,7347,1,22.83,22.83,5,182,117,5,2025,Novembro
617,9762,2,43.6,87.2,12,640,285,18,2025,Novembro
856,10523,2,21.69,43.38,6,220,174,37,2024,Janeiro
946,10696,1,22.14,22.14,3,535,150,19,2024,Julho
534,14640,1,44.21,44.21,12,252,285,20,2024,Dezembro
